# X-Gradient Boosting with CS:GO Data
Predictions at the end...

In [22]:
import xgboost as xgb
import os
import numpy as np
import pandas as pd
import math

# make some function:
def make_ranks(r1, r2):
    # take in ranks for t1 then get difference
    for i in range(len(r1)):
    # see if each one is nan...
        if math.isnan(r1[i]):
            r1[i] = 5
        elif r1[i] > 5:
            r1[i] = 5
        
    for i in range(len(r2)):
    # see if each one is nan...
        if math.isnan(r2[i]):
            r2[i] = 5
        elif r1[i] > 5:
            r2[i] = 5   
    
    return np.reshape(r1 - r2, (len(r1),1))


In [25]:
# change directory to where the PP arrays are:
os.chdir('/home/max/Desktop/Run model')

# load in from preprocessed numpy arrays:
auxillary_train_data = np.load('auxillary_train_data.npz') # Auxillary data for training
auxillary_test_data = np.load('auxillary_test_data.npz') # Auxillary data for testing

# train data
Map_train = auxillary_train_data['arr_0']
Time_train = auxillary_train_data['arr_1']
# reshape time vector to make 2D
Time_train = Time_train.reshape(len(Time_train),1)

t1_roster_train = auxillary_train_data['arr_2']
t2_roster_train = auxillary_train_data['arr_3']

#ranks
rank1train = auxillary_train_data['arr_4']
rank2train = auxillary_train_data['arr_5']
rank1test = auxillary_test_data['arr_4']
rank2test = auxillary_test_data['arr_5']

rankdiff_train = make_ranks(rank1train,rank2train)
rankdiff_test = make_ranks(rank1test,rank2test)

# test data
Map_test = auxillary_test_data['arr_0']
Time_test = auxillary_test_data['arr_1'] 
# reshape time vector to make 2D
Time_test = Time_test.reshape(len(Time_test),1)

t1_roster_test = auxillary_test_data['arr_2']
t2_roster_test = auxillary_test_data['arr_3']

# concatenating testing and training data:
print('map train ',np.shape(Map_train))
print('time train ',np.shape(Time_train))
print('roster 1 train ',np.shape(t1_roster_train))
print('roster 2 train ',np.shape(t2_roster_train))
print('map test ',np.shape(Map_test))
print('time test ',np.shape(Time_test))
print('roster 1 test ',np.shape(t1_roster_test))
print('roster 2 test ',np.shape(t2_roster_test))
print('rankdiff train',np.shape(rankdiff_train))
print('rankdiff test',np.shape(rankdiff_test))

train_inputs = np.concatenate((Time_train,t1_roster_train,t2_roster_train,rankdiff_train),axis=1)

print('train shape:', np.shape(train_inputs))
train_targets = np.load('targets_train.npz')['arr_0']

test_inputs =  np.concatenate((Time_test,t1_roster_test,t2_roster_test,rankdiff_test),axis=1)
print('test shape:', np.shape(test_inputs))
test_targets = np.load('targets_test.npz')['arr_0']

map train  (5479, 9)
time train  (5479, 1)
roster 1 train  (5479, 1413)
roster 2 train  (5479, 1413)
map test  (5249, 9)
time test  (609, 1)
roster 1 test  (609, 1413)
roster 2 test  (609, 1413)
rankdiff train (5479, 1)
rankdiff test (609, 1)
train shape: (5479, 2828)
test shape: (609, 2828)


In [30]:
# Need to remove where the targets == 2.0, otherwise throws off results

# Train data:
dels = [] # array to store deletion indexs:
for i in range(len(train_targets)):
    if train_targets[i] == 2:
        dels.append(i)
# Now remove rows:
train_targets = np.delete(train_targets, dels, 0)
train_inputs = np.delete(train_inputs, dels, 0)

# Testdata:
dels = [] # array to store deletion indexs:
for i in range(len(test_targets)):
    if test_targets[i] == 2:
        dels.append(i)
        
# Remove the rows again:
test_targets = np.delete(test_targets, dels, 0)
test_inputs = np.delete(test_inputs, dels, 0)


In [67]:
# MAKE & RUN THE MODEL

# combine inputs and targets into DMatrix for xgboost:
dtrain = xgb.DMatrix(train_inputs, label = train_targets)
dtest = xgb.DMatrix(test_inputs, label = test_targets)

# specify parameters via map:
# chucked heaps of regularization to prevent overfitting
# plus a binary crossentropy loss
param = {'max_depth':10, 'eta':0.1, 'silent':0, 'min_child_weight':2 ,'objective':'binary:logistic' ,'subsample' : 0.75}
num_round = 30
bst = xgb.train(param, dtrain, num_round)

# make predictions:
preds = bst.predict(dtrain)

In [68]:
# team names - use the train set because data is not aligned correctly:

t1 = auxillary_train_data['arr_6']
t2 = auxillary_train_data['arr_7']

In [69]:
# win predict
def winloss(num):
    if num >= 0.5:
        return 1
    else:
        return 0
    
win_pred = [ winloss(i) for i in preds]
# make the dataframe to display
fram = pd.DataFrame(preds, columns =['Prediction'])
fram['Real Result'] = [int(i) for i in train_targets]
fram['team1'] = t1
fram['team2'] = t2
fram['Predicted Result'] = win_pred


# calculate test set accuracy 
set_acc = np.sum([1 for i,c in enumerate(train_targets) if win_pred[i] == train_targets[i]])/len(train_targets)
print('~5500 games accuracy =', set_acc)

fram

test set (600 games) accuracy = 0.669465230882


,Prediction,Real Result,team1,team2,Predicted Result
0,0.445715,0,iGame.com,ENCORE,0
1,0.407620,0,iGame.com,ENCORE,0
2,0.590918,1,iGame.com,ENCORE,1
3,0.590918,1,Space Soldiers,Gambit,1
4,0.477976,0,Space Soldiers,Gambit,0
5,0.477976,0,ajuri,SkitLite,0
6,0.445919,0,SkitLite,ENCORE,0
7,0.445919,0,SkitLite,ENCORE,0
8,0.437740,0,ajuri,iGame.com,0
9,0.437740,1,ajuri,iGame.com,0


In [70]:
# save as a csv file to look at
fram.to_csv('xgboost_predictions.csv')